In [1]:
import scipy.io as sio
import pandas as pd
from pandas import Series, DataFrame
from sklearn.preprocessing import Imputer
import numpy as np
import xgboost as xgb
from xgboost import plot_importance
from sklearn.model_selection import train_test_split
from sklearn import model_selection, metrics
# from sklearn.grid_search import GridSearchCV
from xgboost.sklearn import XGBClassifier
import sys
import copy

In [2]:
# 类别映射
idx = [2, 3, 5, 6, 8, 10, 11, 12, 14]
_idx = {}
j = 0
for i in idx:
    _idx[i] = j
    j += 1
_idx

{2: 0, 3: 1, 5: 2, 6: 3, 8: 4, 10: 5, 11: 6, 12: 7, 14: 8}

In [3]:
# read file
data_x = []
data_y = []
test_x = sio.loadmat('data_test_final.mat')
test_x = test_x['data_test_final']
tot = 0
for i in range(9): 
    key = 'data' + str(idx[i]) + '_train'
    tmp = sio.loadmat(key + '.mat')
    tmp = tmp[key]
    length = len(tmp)
    print(length)
    tot += length
    for j in range(length):
        data_x.append(tmp[j])
        data_y.append(i)
print(tot)

1071
622
362
547
358
729
1841
445
949
6924


In [4]:
def fun(par,num_rounds):
    x_train, x_valid, y_train, y_valid = train_test_split(data_x, data_y, test_size=0.3)
    Dtrain = xgb.DMatrix(x_train, y_train)
    # train model
    plst=par.items()
    model=xgb.train(plst,Dtrain,num_rounds)

    # predict valid data set
    dvalid = xgb.DMatrix(x_valid)
    ans = model.predict(dvalid)
    dvalid = xgb.DMatrix(x_valid)
    ans = model.predict(dvalid)

    # calculate the F1 score
    length = len(y_valid)
    tp = tn = fp = fn = cnt = 0
    for i in range(length):
        p,t = ans[i],y_valid[i]
        if p == 1 and t == 1:
            tp += 1
        elif p == 0 and t == 0:
            tn += 1
        elif p == 1 and t == 0:
            fp += 1
        elif p == 0 and t == 1:
            fn += 1
    pp = tp / (tp + fp)
    rr = tp / (tp + fn)
    f1 = 2 * (pp * rr) / (pp + rr)

    # calculate the precision
    #for i in range(length):
    #    if ans[i] == y_valid1[i]:
    #        cnt += 1
    #precision = cnt / length
    # print(precision)
    
    test = xgb.DMatrix(test_x)
    final = np.array(model.predict(test),dtype='int64')
    length = len(final)
    for i in range(length):
        final[i] = idx[final[i]]
    final = pd.DataFrame({'y':final})
    file = pd.concat([final],sort=False)
    file.to_csv("ans.csv",index=False,sep=',')

    return f1, pp, rr
params = {
    'booster': 'gbtree',
    'objective': 'multi:softmax',
    'num_class': 9,
    'gamma': 0,# 0 ~ 1 0.5 3
    'max_depth': 5,# 5 ~ 7 3
    'lambda': 1,
    'subsample': 0.7,# 0.6 ~ 0.8 0.1 3
    'colsample_bytree': 0.7,# 0.6 ~ 0.8 0.1 3
    'min_child_weight': 3, # 2 ~ 4 1 3
    'eta': 0.1, # 0.09 ~ 0.11 0.01 3
#    'seed': 1000,
#    nthread : 4
    'silent': 1,
}
fun(params, 200)

(0.9776357827476039, 0.9935064935064936, 0.9622641509433962)

In [5]:
print(len(set(data_y)))

9
